In [193]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## A brief introduction to Algorithmic Music Composition (in Python!)

### First, let's import some modules.

In [194]:
import numpy as np # NumPy allows us to do math with arrays.  
import cosmos # a custom library for working with audio
path = './'
# path = 'cosmos/'

In [195]:
# load an audio sample
snare = cosmos.load_audio(path + "snare.wav")
# and listen to it
cosmos.play_audio(snare)


Max amplitude was 1.000030517578125, normalizing.


In [196]:
# create an empty audio track
track = cosmos.AudioTrack(duration=5)
# add the snare sample to it
track.add_audio(snare, time=0.0) # at time 0 (in seconds)
track.add_audio(snare, time=1.0) # and at time 1
cosmos.play_audio(track)

In [197]:
# clear the audio track
track.clear()
cosmos.play_audio(track)

In [198]:
track.clear()
# add the snare sample at specified times
times = [0, 0.25, 0.75, 1.25, 1.5, 1.625, 2, 2.125]
# use a for loop to add the snare sample at each time
for time in times:
    track.add_audio(snare, time=time)

cosmos.play_audio(track)

In [199]:
track.clear()
# set the gain of the snare sample (ranging from 0 to 1)
# so that it gets quieter over time
track.add_audio(snare, time=0.0, gain=1.0)
track.add_audio(snare, time=0.25, gain=0.5)
track.add_audio(snare, time=0.5, gain=0.25)
track.add_audio(snare, time=0.75, gain=0.125)
track.add_audio(snare, time=1.0, gain=0.0625)
track.add_audio(snare, time=1.25, gain=0.03125)
track.add_audio(snare, time=1.5, gain=0.015625)
track.add_audio(snare, time=1.625, gain=0.0078125)
cosmos.play_audio(track)

In [200]:
# Same thing, but with more efficient code
track.clear()
# set the array of times using a list comprehension
# this is equivalent to building the array using a for loop:
''' 
times = []
 for i in range(8):
    times.append(i/4)
'''
times = [i/4 for i in range(8)] # [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75]

# set the array of gains using a list comprehension
# this is equivalent to building the array using a for loop:
'''
gains = []
for i in range(8):
    gains.append(2 ** -i)
'''
gains = [2 ** -i for i in range(8)] # [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]

for i in range(len(times)):
    track.add_audio(snare, time=times[i], gain=gains[i])
cosmos.play_audio(track)

In [201]:
track.clear()
# faster, longer, slower decay
times = [i/14 for i in range(32)]
gains = [2 ** (-0.25 * i) for i in range(32)]

print([round(i, 2) for i in gains])
for i in range(len(times)):
    track.add_audio(snare, time=times[i], gain=gains[i])
cosmos.play_audio(track)

[1.0, 0.84, 0.71, 0.59, 0.5, 0.42, 0.35, 0.3, 0.25, 0.21, 0.18, 0.15, 0.12, 0.11, 0.09, 0.07, 0.06, 0.05, 0.04, 0.04, 0.03, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0]


## Patterns

In [202]:
# Create a pattern of times (using durations) and gains
track = cosmos.AudioTrack(duration=20)

durations = [1/2, 1/4, 1/2, 1/4, 1/8, 3/8]
# get the times by cumulatively summing the durations
# times = [sum(durations[:i]) for i in range(len(durations))] 
gains = [0.3, 0.7, 0.3, 1, 0.7, 0.3]
pattern = cosmos.Pattern(durations=durations, gains=gains, samples=snare, cycles=4)
track.add_audio(pattern.to_audio())
cosmos.play_audio(track)


In [203]:
# vary the samples
track.clear()

hihat = cosmos.load_audio(path + "hihat.wav")
samples_1 = [hihat, hihat, snare, hihat, hihat, snare]
pattern = cosmos.Pattern(durations=durations, gains=gains, samples=samples, cycles=4)
track.add_audio(pattern.to_audio(), time=0.0)
cosmos.play_audio(track)

In [204]:
# create a new pattern, and alternate between the two patterns
track.clear()

pattern_1 = cosmos.Pattern(durations=durations, gains=gains, samples=samples_1, cycles=2)
durations_2 = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 3/8, 1/4, 3/8]
hihat = cosmos.load_audio(path + "hihat.wav")
bongo = cosmos.load_audio(path + "bongo.wav")
samples_2 = [bongo, bongo, bongo, bongo, bongo, snare, snare, snare, snare, hihat, hihat]
gains_2 = [1, 0.9, 0.8, 0.7, 0.6, 0.3, 0.4, 0.5, 0.6, 1, 1]
pattern_2 = cosmos.Pattern(durations=durations_2, gains=gains_2, samples=samples_2, cycles=2)
# add the first pattern
track.add_audio(pattern_1.to_audio(), time=0.0)
next_time = pattern_1.dur_tot
# add the second pattern
track.add_audio(pattern_2.to_audio(), time=next_time)
next_time += pattern_2.dur_tot
# add the first pattern again
track.add_audio(pattern_1.to_audio(), time=next_time)
cosmos.play_audio(track)


Max amplitude was 1.000030517578125, normalizing.
Audio would have clipped, had to normalize it first.


In [205]:
track.clear()
# alter the total duration of a pattern

# first add the normal pattern
pattern_2 = cosmos.Pattern(durations=durations_2, gains=gains_2, samples=samples_2, cycles=2)
track.add_audio(pattern_2.to_audio(), time=0.0)
next_time = pattern_2.dur_tot
# then change the total duration of the pattern, making it twice as fast
pattern_2.set_cycle_duration(pattern_2.cycle_dur / 2)
# and extend the number of repetitions
pattern_2.cycles *= 2
# add the pattern again
track.add_audio(pattern_2.to_audio(), time=next_time)
cosmos.play_audio(track)

Audio would have clipped, had to normalize it first.


# Working with Meter

In [206]:
# define a "4/4" meter with 4 beats per measure and each beat subdivided into two parts
# tempo of 60 bpm, for 4 cycles
simple_meter = cosmos.Meter(hierarchy=[4, 2], tempo=120, cycles=4)
all_subdivisions = simple_meter.all_times()
all_pulses = simple_meter.all_times(top_layer=1)
print('all_pulses', '\n', all_pulses)
print('\n', 'all_subdivisions', '\n', all_subdivisions)




all_pulses 
 [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5]

 all_subdivisions 
 [0.   0.25 0.5  0.75 1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.   3.25
 3.5  3.75 4.   4.25 4.5  4.75 5.   5.25 5.5  5.75 6.   6.25 6.5  6.75
 7.   7.25 7.5  7.75]


In [207]:
track = cosmos.AudioTrack(duration=20)
kick = cosmos.load_audio(path + "kick.wav")
# add hi-hat every subdivision
cosmos.play_audio(track)



In [208]:
track.clear()
# set up basic drum track
for time in all_subdivisions: track.add_audio(hihat, time=time, gain=0.5)
kick_times = [item for i, item in enumerate(all_pulses) if i % 2 == 0]
for time in kick_times: track.add_audio(kick, time=time, gain=1.0)
snare_times = [item for i, item in enumerate(all_pulses) if i % 2 == 1]
for time in snare_times: track.add_audio(snare, time=time, gain=1.0)

# now add bongo hits one by one
# remember: the first beat is beat 0, the first subdivision is subdivision 0!

#beat 2, subdivision 2 
time = simple_meter.get_time([1, 1])
track.add_audio(bongo, time=time)
# beat 3, subdivision 2
time = simple_meter.get_time([2, 1])
track.add_audio(bongo, time=time)
# beat 4, subdivision 1
time = simple_meter.get_time([3, 0])
track.add_audio(bongo, time=time)
cosmos.play_audio(track)



Audio would have clipped, had to normalize it first.


In [210]:
track.clear()
# set up basic drum track
kick_times = [item for i, item in enumerate(simple_meter.all_times(top_layer=1)) if i % 2 == 0]
snare_times = [item for i, item in enumerate(simple_meter.all_times(top_layer=1)) if i % 2 == 1]
for time in simple_meter.all_times(): track.add_audio(hihat, time=time, gain=0.5)
kick_times = [item for i, item in enumerate(all_pulses) if i % 2 == 0]
for time in kick_times: track.add_audio(kick, time=time, gain=1.0)
snare_times = [item for i, item in enumerate(all_pulses) if i % 2 == 1]
for time in snare_times: track.add_audio(snare, time=time, gain=1.0)

# now add some bongo hits by setting up the pulse / subdivisions in adavnce
bongo_hits = [[1, 1], [2, 1], [3, 0], [4, 0], [4, 1], [5, 1], [6, 1], [7, 0]]
for hit in bongo_hits:
    time = simple_meter.get_time(hit)
    track.add_audio(bongo, time=time)
cosmos.play_audio(track)

Audio would have clipped, had to normalize it first.
